# VAE Training

## imports

In [23]:
import os, time
from images.Metrics import calculate_metrics
import numpy as np

from model.VariationalAutoencoder import VariationalAutoencoder

from keras.preprocessing.image import ImageDataGenerator, load_img, save_img, img_to_array, array_to_img

# environment variables

In [24]:
MODEL_FOLDER = os.environ.get('HOME') + '/model3/weights'
FILE_MODEL = 'weights.h5'
DATA_FOLDER = '/Volumes/My Passport/PFM/output/training_png'
RUN_FOLDER =  '/Volumes/My Passport/PFM/run/'

# VAE

## model variables

In [25]:
vae = VariationalAutoencoder(image_folder=DATA_FOLDER, run_folder=RUN_FOLDER, train_mode= False, z_dim=200, use_dropout=False)
vae.model.load_weights(os.path.join(MODEL_FOLDER, 'weights.h5'))  

In [27]:
from PIL import Image
from PIL.ImageShow import show

#image_path = '/Volumes/My Passport/PFM/output/training_png/Week9_39301_G11_4_Cells_34.png'
image_path = '/Volumes/My Passport/PFM/output/training_png/Week5_28921_B02_2_Cells_13.png'
#image_path = '/Volumes/My Passport/PFM/output/training_png/Week5_28921_B02_3_Cells_39.png'
#image_path = '/Volumes/My Passport/PFM/output/training_png/Week7_34641_B02_1_Cells_28.png'

img = Image.open(image_path)
array_img  = img_to_array(img) # keras



array_img = array_img / 255.

print("max: " + str(np.amax(array_img)))

#print(array_img)

img.show()

print('image format: ' + str(img.format))
#print('image shape: ' + str(img.shape))
print('image mode: ' + img.mode)
print('image size: ' + str(img.size))
print('image width: ' + str(img.width))
print('image height: ' + str(img.height))
print('image palette: ' + str(img.palette))
print('image info: ' + str(img.info))

print('image to array: ' + str(array_img.shape))

array_img = np.expand_dims(array_img, axis=0)


vae.encoder.summary()

print('image to array after expand: ' + str(array_img.shape))

z_result = vae.encoder.predict(array_img)
reco = vae.decoder.predict(z_result).squeeze()

#print(reco)

print(reco.shape)

# scaled
outimg = array_to_img(reco, scale=255.)

show(outimg)

print(reco)


max: 0.47058824
image format: PNG
image mode: RGB
image size: (128, 128)
image width: 128
image height: 128
image palette: None
image info: {}
image to array: (128, 128, 3)
Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, 64, 64, 32)   128         encoder_conv_1[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_31 (L

# Metrics 

In [ ]:
calculate_metrics(vae = vae, path = DATA_FOLDER)